# Dataextraction from openlegaldata
To create our training and test datasets, we scraped the data from the Open Legal Data platform. Initially, we attempted to use the official API, but it did not provide the results we needed. Therefore, we implemented a custom Python solution using the requests library to reliably retrieve the required data. Our idea was to first create a training dataset that the model could use for self-supervised learning, allowing it to learn the underlying structure and relationships within legal texts without requiring labeled data. After this initial phase, we planned to construct a supervised dataset consisting of legal cases paired with the specific laws applied in each decision. This labeled dataset would then be used to evaluate whether the model can correctly identify the relevant legal provisions, enabling us to measure its accuracy and overall performance.

In [2]:
# import necessary libraries
import oldp_client
import requests
import re
import csv
from bs4 import BeautifulSoup
import pandas as pd
import time

In [13]:
API_KEY = ""

In [14]:
headers = {
    "Authorization": f"Token {API_KEY}"
}

## Test
To verify that our data extraction approach works correctly, we performed an initial test request using the Open Legal Data API. The following snippet retrieves a small sample of five cases, selects the first entry, and extracts its HTML content. We then use BeautifulSoup to convert the HTML into clean plain text

In [15]:
# test try
url = "https://de.openlegaldata.io/api/cases/?limit=5"
resp = requests.get(url)
data = resp.json()

case = data["results"][0]
raw_html = case.get("content") or ""

soup = BeautifulSoup(raw_html, "html.parser")
plain_text = soup.get_text(separator="\n", strip=True)

print("Case ID:", case["id"])
print("Plain text excerpt:\n")
print(plain_text)

Case ID: 325566
Plain text excerpt:

Tenor
I. Die Beklagte wird verurteilt, der Klägerin vollständige schriftliche Auskunft darüber zu erteilen, in welchem Umfang die Beklagte im geschäftlichen Verkehr in Deutschland Hosen wie nachfolgend eingeblendet
(Es folgt eine Darstellung)
angeboten und/oder beworben und/oder vertrieben hat und zwar insbesondere durch Vorlage eines Verzeichnisses, aus dem sich je Produktart ergibt:
- Namen und Anschrift der gewerblichen Abnehmer der oben abgebildeten Hosen und Verkaufsstellen, für die sie bestimmt waren bzw. an die sie geliefert worden sind; und
- die Menge der ausgelieferten, erhaltenen oder bestellten oben abgebildeten Hosen sowie die Preise, die für diese von ihren Abnehmern bezahlt wurden, sowie die entsprechenden Einstandspreise für die Produkte;
- der Umfang der für die oben abgebildeten Hosen betriebenen Werbung, unter Mitteilung der Werbemedien und ihrer Erscheinungsdaten und Auflagenzahlen, Sendedaten und -reichweiten, Veröffentlichungen

## Extracting laws
To extract the legal citations for our test dataset, we processed the HTML content of each case using BeautifulSoup to obtain plain text. We then applied a custom regular expression designed to detect German legal references, including both single and multiple sections (§, §§) as well as typical law abbreviations (e.g., BGB, StGB, ZPO). The following code shows this extraction step

In [16]:
soup = BeautifulSoup(raw_html, "html.parser")
text = soup.get_text(separator=" ")

pattern = r"""
(§{1,2}              # § or §§
\s*
[\d,\s\-–abcfIVX]+   # numbers, letters, ranges, multiple sections
\s*
([A-ZÄÖÜ]{2,10})     # law abbreviation (ZPO, BGB, StGB, GG, etc.)
)
"""

matches = re.findall(pattern, text, flags=re.VERBOSE)

# Clean
citations = [m[0].strip() for m in matches]
print(citations)

['§ 242 BGB', '§ 203 BGB', '§§ 91, 709 ZPO']


## Fetching
To collect a larger number of cases for our dataset, we implemented a function that iteratively retrieves cases from the Open Legal Data API by following the pagination links. The function continues requesting pages until the desired number of cases is reached:

In [18]:
def fetch_cases(n):
    cases = []
    url = "https://de.openlegaldata.io/api/cases/?offset=0"

    while url and len(cases) < n:
        resp = requests.get(url).json()
        cases.extend(resp["results"])
        
        url = resp.get("next")  # next page
        
    return cases[:n]

cases_100 = fetch_cases(100)
print("Fetched:", len(cases_100))

Fetched: 100


After testing our initial case-fetching function, we realized that simply retrieving the raw API data was not sufficient for building a high-quality dataset. The case texts provided by the API are embedded in HTML, which makes direct processing and law-extraction unreliable.
To address this, we implemented an improved version of the function that not only fetches the cases but also cleans and standardizes them immediately:

In [3]:
def fetch_cases_clean(n):
    cases = []
    url = "https://de.openlegaldata.io/api/cases/?offset=0"

    while url and len(cases) < n:
        resp = requests.get(url).json()
        cases.extend(resp["results"])
        url = resp.get("next")

    cases = cases[:n]

    cleaned_cases = []

    for case in cases:
        raw_html = case.get("content", "")
        soup = BeautifulSoup(raw_html, "html.parser")
        plain_text = soup.get_text(separator="\n", strip=True)

        cleaned_cases.append({
            "id": case.get("id"),
            "court": case.get("court", {}).get("name"),
            "date": case.get("date"),
            "text": plain_text
        })

    return cleaned_cases

# Fetch 1200 cases total
cases_all = fetch_cases_clean(1200)
train_cases = cases_all[:1000]
test_cases  = cases_all[1000:1200]
print("fertig")

fertig


In [22]:
# blocked for 24h wen trying a second time
resp = requests.get(url).json()
print(resp)   # <-- debug

{'message': 'Die Anfrage wurde gedrosselt. Expected available in 84766 seconds.', 'code': 'throttled'}


## Better law pattern
To improve the accuracy of our law extraction, we refined our citation-matching logic by creating a more comprehensive regular expression. This updated pattern now captures a wide range of German legal references, including sections with multiple subsections, sequences of citations, and combinations using connectors such as i.V.m., und, or sowie. We also extract the corresponding law abbreviation (e.g., BGB, VwGO, StGB).

The extraction step was then applied directly to our cleaned test dataset and then integrated into an extra column.

In [4]:
pattern = r"""
(
    §{1,2}\s*\d+[a-zA-Z]*                          
        (?:\s*(?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+)*

    (?:                                 
        \s*,\s*
        (?:
            (?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+     
                (?:\s*(?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+)*
            |
            \d+[a-zA-Z]*                           
                (?:\s*(?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+)*
        )
    )*

    (?:
        \s*(?:i\.?V\.?m\.?|in\s+Verbindung\s+mit|und|sowie|oder)\s*
        (?:
            §{1,2}\s*\d+[a-zA-Z]*                  
                (?:\s*(?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+)*
            |
            (?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+     
                (?:\s*(?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+)*
            |
            \d+[a-zA-Z]*                           
                (?:\s*(?:Abs\.?|Absatz|Satz|Nr\.?)\s*\d+)*
        )
    )*

    \s+[A-Za-zÄÖÜäöü]{2,15}
)
"""

def extract_citations(text):
    text = text.replace("\xa0", " ")  # NBSP bereinigen
    matches = re.findall(pattern, text, flags=re.VERBOSE | re.IGNORECASE)
    return [m.strip() for m in matches]

for case in test_cases:
    case["citations"] = extract_citations(case["text"])


print("Fertig!")
print("Beispiel:")
print(test_cases[0])

Fertig!
Beispiel:
{'id': 345731, 'court': 'Verwaltungsgericht Göttingen', 'date': '2022-06-17', 'text': 'Tatbestand\n1\nDer Kläger begehrt die Flüchtlingsanerkennung, hilfsweise den subsidiären Schutzstatus, weiter hilfsweise die Zuerkennung von Abschiebungsverboten.\n2\nDer 31 Jahre alte Kläger ist russischer Staatsangehöriger und von Beruf Ingenieur. Er reiste mit einem deutschen Visum nach eigenen Angaben über Polen kommend am 01.08.2018 in die Bundesrepublik Deutschland ein. Erst am 08.01.2020 stellte er einen Asylantrag.\n3\nIm Rahmen seiner persönlichen Anhörung durch das Bundesamt für Migration und Flüchtlinge (im Folgenden: Bundesamt) am 22.07.2020 trug der Kläger im Wesentlichen vor, er habe die Russische Föderation aus Sorge vor Verfolgung wegen seiner antifaschistischen Aktivitäten vor allem in sozialen Medien verlassen. Er habe seit dem Jahr 2009 der Antifaschismusbewegung angehört und sei zunächst zu Demonstrationen gegangen. In dieser Zeit sei er mehrfach von der Polizei 

## Write CSV
Finally we saved the datasets into CSV to load them later into our LLM's.

In [5]:
with open("training_cases.csv", "w", newline="", encoding="utf-8") as fp:

    # Get column names from the first entry
    fieldnames = ["id", "court", "date", "text"]

    writer = csv.DictWriter(
        fp,
        fieldnames=fieldnames,
        delimiter=";",      # semicolon separator
        quoting=csv.QUOTE_MINIMAL
    )

    writer.writeheader()

    # Write each case as a separate row
    for case in train_cases:
        writer.writerow(case)

print("CSV successfully written.")

CSV successfully written.


In [6]:
# Increase max field size to get the long cases
csv.field_size_limit(10_000_000)

with open("training_cases.csv", "r", encoding="utf-8") as fp:
    reader = csv.DictReader(fp, delimiter=";")
    rows = list(reader)

print(rows[0])

{'id': '325566', 'court': 'Landgericht Köln', 'date': '2029-11-13', 'text': 'Tenor\nI. Die Beklagte wird verurteilt, der Klägerin vollständige schriftliche Auskunft darüber zu erteilen, in welchem Umfang die Beklagte im geschäftlichen Verkehr in Deutschland Hosen wie nachfolgend eingeblendet\n(Es folgt eine Darstellung)\nangeboten und/oder beworben und/oder vertrieben hat und zwar insbesondere durch Vorlage eines Verzeichnisses, aus dem sich je Produktart ergibt:\n- Namen und Anschrift der gewerblichen Abnehmer der oben abgebildeten Hosen und Verkaufsstellen, für die sie bestimmt waren bzw. an die sie geliefert worden sind; und\n- die Menge der ausgelieferten, erhaltenen oder bestellten oben abgebildeten Hosen sowie die Preise, die für diese von ihren Abnehmern bezahlt wurden, sowie die entsprechenden Einstandspreise für die Produkte;\n- der Umfang der für die oben abgebildeten Hosen betriebenen Werbung, unter Mitteilung der Werbemedien und ihrer Erscheinungsdaten und Auflagenzahlen, S

In [7]:
# test if the csv is correct
df = pd.read_csv("training_cases.csv", sep=";", encoding="utf-8")

df.head()   # shows first 5 rows as a table

,id,court,date,text
0,325566,Landgericht Köln,2029-11-13,"Tenor\nI. Die Beklagte wird verurteilt, der Kl..."
1,323770,Sozialgericht Düsseldorf,2027-04-06,Tenor\nDie Klage wird abgewiesen. Außergericht...
2,343880,Landesarbeitsgericht Düsseldorf,2022-12-15,Tenor\n1. Auf die Berufung des Klägers wird da...
3,343848,Amtsgericht Essen,2022-10-14,Tenor\nDer Vollstreckungsbescheid des Amtsgeri...
4,346952,Oberlandesgericht München,2022-10-13,Tenor\nI. Die Berufung der Beklagten gegen das...


In [8]:
# now save the test cases into a csv
with open("test_cases.csv", "w", newline="", encoding="utf-8") as fp:

    # Get column names from the first entry
    fieldnames = ["id", "court", "date", "text","citations"]

    writer = csv.DictWriter(
        fp,
        fieldnames=fieldnames,
        delimiter=";",      # semicolon separator
        quoting=csv.QUOTE_MINIMAL
    )

    writer.writeheader()

    # Write each case as a separate row
    for case in test_cases:
        writer.writerow(case)

print("CSV successfully written.")

CSV successfully written.


In [10]:
df = pd.read_csv("test_cases.csv", sep=";")
print(df.head())

       id                                       court        date  \
0  345731                Verwaltungsgericht Göttingen  2022-06-17   
1  345695  Oberverwaltungsgericht Nordrhein-Westfalen  2022-06-17   
2  345694  Oberverwaltungsgericht Nordrhein-Westfalen  2022-06-17   
3  345693  Oberverwaltungsgericht Nordrhein-Westfalen  2022-06-17   
4  345692  Oberverwaltungsgericht Nordrhein-Westfalen  2022-06-17   

                                                text  \
0  Tatbestand\n1\nDer Kläger begehrt die Flüchtli...   
1  Tenor\nDer angefochtene Beschluss wird geänder...   
2  Tenor\nDie Beschwerde wird zurückgewiesen.\nDe...   
3  Tenor\nDer Antrag des Klägers auf Zulassung de...   
4  Tenor\nDer Antrag der Kläger auf Zulassung der...   

                                           citations  
0  ['§ 60 Abs. 5 und 7 Satz 1 AufenthG', '§ 102 A...  
1  ['§ 81 BauO', '§ 81 Abs. 1 BauO', '§ 81 Abs. 2...  
2  ['§ 146 Abs. 4 Satz 6 VwGO', '§§ 80 Abs. 5, 80...  
3  ['§ 78 Abs. 3 Nr. 1 AsylG

In [11]:
# test for second csv
with open("test_cases.csv", "r", encoding="utf-8") as fp:
    reader = csv.DictReader(fp, delimiter=";")
    first_row = next(reader)          # get the first case

text_only = first_row["text"]         # extract only the text field

print(text_only)

Tatbestand
1
Der Kläger begehrt die Flüchtlingsanerkennung, hilfsweise den subsidiären Schutzstatus, weiter hilfsweise die Zuerkennung von Abschiebungsverboten.
2
Der 31 Jahre alte Kläger ist russischer Staatsangehöriger und von Beruf Ingenieur. Er reiste mit einem deutschen Visum nach eigenen Angaben über Polen kommend am 01.08.2018 in die Bundesrepublik Deutschland ein. Erst am 08.01.2020 stellte er einen Asylantrag.
3
Im Rahmen seiner persönlichen Anhörung durch das Bundesamt für Migration und Flüchtlinge (im Folgenden: Bundesamt) am 22.07.2020 trug der Kläger im Wesentlichen vor, er habe die Russische Föderation aus Sorge vor Verfolgung wegen seiner antifaschistischen Aktivitäten vor allem in sozialen Medien verlassen. Er habe seit dem Jahr 2009 der Antifaschismusbewegung angehört und sei zunächst zu Demonstrationen gegangen. In dieser Zeit sei er mehrfach von der Polizei kontrolliert worden, seine Daten seien aufgenommen worden. Ab 2012 bis 2017 habe er eine eigene Internetseite b

## Conclusion
In the end, we produced two structured datasets:

1. A large, unlabeled corpus of cleaned case texts for self-supervised pretraining, enabling the model to learn
2. A smaller, carefully curated supervised dataset in which all legal citations were automatically extracted and partly verified by hand. This ensures that the labels are accurate and that the evaluation of the model is reliable.

Together, these datasets form an ideal foundation for developing our legal language model. The self-supervised corpus allows the model to build a strong general understanding, while the supervised citation dataset enables precise training and evaluation of the model’s ability to identify relevant laws in real cases.